# The Real code

In [1]:
using Random, Distributions, Plots, XLSX

What needs to be done:
. Statistics along the way
. Rebalancing
... There is a problem with the down time, and the simulation gives a shit ton of unfullfilled values, seems fucked!
. Last simulation
... Plots
... Time to run consecutively (does that have something to do with rebalancing?)
... 

Questions
. Weird arrival at midnight
. Can rebalance time be float?


In [146]:
### Initial functions to load lambda and mu from the ArrivalProfiles.xlsx

# Change the type to Float64 from Int and String for the loaded vectors
function type_change(v)
    v_ny = Vector{Float64}()
    for i in 1:length(v)
        if typeof(v[i]) == String
            push!(v_ny,parse(Float64,v[i]))
        elseif typeof(v[i]) == Int64
            push!(v_ny,float(v[i]))
        else
            push!(v_ny,v[i])
        end
    end
    return v_ny
end


function load_arri(path="C:/Users/rasmu/Desktop/OneDrive/OneDrive - Danmarks Tekniske Universitet/Decisions under uncertainty/Projekt/ArrivalProfiles.xlsx")
    xf = XLSX.readxlsx(path) # Loading

    # Lambda is given as expected arrivals therefore inverse
    lambda_port = 1 ./ type_change(xf[1]["C2:AA2"])
    lambda_dtu = 1 ./ type_change(xf[1]["C3:AA3"])

    # Travel time
    mu_rammer_dtu = type_change(xf[1]["C7:AA7"])
    mu_rammer_port = type_change(xf[1]["C8:AA8"])

    return lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port
end


# Might not be just as fast as PriorityQueue but it is not that much behind
# Inserts a value at the spot it fits
function insert_and_sort!(vect::Vector, x)
    (splice!(vect, searchsorted(vect,x), [x]); vect) 
end


insert_and_sort! (generic function with 1 method)

In [153]:
struct Environment # The environment including arrival rates and travel rates
    lambda_dtu::Vector{Float64}  #arrival rate 1
    mu_rammer_dtu::Vector{Float64} #service rate 1
    lambda_port::Vector{Float64} # arrival rate 2
    mu_rammer_port::Vector{Float64} #service rate 2
    
    # loading the vectors from excel
    lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port = load_arri() # NEEDS the path to load the document from!
    Environment() = new(lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port)
end

mutable struct Stats # Vector including [Station 1, Station 2]
    n_cars::Vector{Int64} # Number of cars at the stations at any given time
    un_fullfilled::Vector{Int64} # Amount of times a customer arrives to an empty station
    fullfilled::Vector{Int64} # Amount of times a costumer arrives and gets a car
    empty_time::Vector{Float64} # The time any given station has 0 cars
end

struct Balance
    desired_n_st1::Int64
    desired_n_st2::Int64
    time_1::Float64
    time_2::Float64
end


In [154]:
# Do this:
# stats.overview_usage = [stats.overview_usage; t state.queue_length state.server_busy]
# Real nice, steal that shit!

function time_change(e::Array{Tuple{Float64,String},1},k::Int64,day::Int64) # problems
    # Switch day and reset index
    if k == 25
        k = 0
        day += 25
    end
    # Insert the next time change
    popfirst!(e)[1]
    insert_and_sort!(e,(k+day,"time_change"))

    # Change index for time
    k += 1
    return k,day
end

function arrival(station::Int64,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats,k::Int64)
    
    # Insert next arrival based on previous arrival
    if station == 1
        insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k]))+e[1][1],"st1"))
    elseif station == 2
        insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k]))+e[1][1],"st2"))
    end

    if stat.n_cars[station] == 0 # Student arrives at empty station. That is one unsatisfied costumer :(
        stat.un_fullfilled[station] += 1 ################################################################################################## THe problem is the timer!!! It will take minus every time but only plus once§!!!!!!!!!!!!!
        popfirst!(e)
    else # Costumer leaves station
        stat.n_cars[station] -= 1 # removing car from station
        stat.fullfilled[station] += 1 # one fullfilled request
        
        # Insert travel time based on arrival station
        if station == 1
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2")) 
        elseif station == 2
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st1")) 
        end

        # Removing the occured event
        if stat.n_cars[station] == 0 # station becomes empty
            stat.empty_time[station] -= popfirst!(e)[1] # Empty station timer starts 
        else
            popfirst!(e)
        end
    end 

end 

function travel(station::Int64,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats)

    if stat.n_cars[station] == 0 # Arriving at an empty station
        stat.n_cars[station] += 1 # arrive
        stat.empty_time[station] += popfirst!(e)[1] # End empty timer
    else
        stat.n_cars[station] += 1 # arrive
        popfirst!(e)
    end

end


function rebalancing(rebal::Balance,stat::Stats,env::Environment,e::Array{Tuple{Float64,String},1},k::Int64)
    # NEEDS Documentation and a serious debugging
    if stat.n_cars[2] >= rebal.desired_n_st1
        for i in 1:rebal.desired_n_st1
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1") ) 
            stat.n_cars[2] -= 1
        end
        
        # Start empty timer
        if stat.n_cars[2] == 0 
            stat.empty_time[2] -= e[1][1]
        end

    else
        for i in 1:stat.n_cars[2]
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1") )
            stat.n_cars[2] -= 1
        end
        stat.empty_time[2] -= e[1][1]

    end

    # For station 1
    if stat.n_cars[1] >= rebal.desired_n_st2
        for i in 1:rebal.desired_n_st2
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2") )
            stat.n_cars[1] -= 1
        end
        
        # Start empty timer
        if stat.n_cars[1] == 0 
            stat.empty_time[1] -= e[1][1]
        end
    else
        for i in 1:stat.n_cars[1]
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2") )
            stat.n_cars[1] -= 1
        end
        stat.empty_time[1] -= e[1][1]
    end 

    insert_and_sort!(e,(24.0+popfirst!(e)[1],"rebalance") )
    
end


rebalancing (generic function with 2 methods)

In [155]:
function sim(days_to_run::Int64, rebal::Balance, env::Environment,stat::Stats)
    # Remember to do something with rebalance!!!!!!!!!!!!
    e = Array{Tuple{Float64,String},1}()
    
    runs = 0
    k = 1
    day = 0
    insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k])),"st1")) # Insert first arrival
    insert_and_sort!(e,(rand(Exponential(env.lambda_port[k])),"st2")) # Insert frist arrival
    insert_and_sort!(e,(0.0,"time_change")) # THIS NEEDS TO BE FIXED!!!!!!!!

    insert_and_sort!(e,(rebal.time_1,"rebalance"))
    insert_and_sort!(e,(rebal.time_2,"rebalance"))

    while e[1][1] <= days_to_run*24.0 && runs <= 10000
        load_type = e[1][2]
        if load_type == "st1"
            arrival(1,e,env,stat,k)
        elseif load_type == "st2"
            arrival(2,e,env,stat,k)
        elseif load_type == "que_st1"
            travel(1,e,env,stat)
        elseif load_type == "que_st2"
            travel(2,e,env,stat)
        elseif load_type == "time_change"
            k,day = time_change(e,k,day)
        elseif load_type == "rebalance"
            rebalancing(rebal,stat,env,e,k)
        else
            throw(ErrorException("Issue in the event type!"))
        end
        runs += 1 # Checks the amount of events
    end
    
    # Make sure the empty timer stops at the end of the run
    if stat.n_cars[1] == 0
        stat.empty_time[1] += days_to_run*24.0
    elseif stat.n_cars[2] == 0
        stat.empty_time[2] += days_to_run*24.0
    end
    
    return stat.n_cars,stat.un_fullfilled,stat.fullfilled,stat.empty_time,runs
end

sim (generic function with 4 methods)

### Testing

In [179]:

function main(n_cars::Vector{Int64},days_to_run::Int64,n_of_runs::Int64,rebalance::Vector,path = "C:/Users/rasmu/Desktop/OneDrive/OneDrive - Danmarks Tekniske Universitet/Decisions under uncertainty/Projekt/ArrivalProfiles.xlsx")
    # Rebalance is given in [desired_n_st1, desired_n_st2, time_1, time_2] for now int
    env = Environment()
    stat = Stats(n_cars,[0,0],[0,0],[0,0])
    rebal = Balance(rebalance[1],rebalance[2],rebalance[3],rebalance[4])
    
    for i in 1:n_of_runs
        stat = Stats(n_cars,[0,0],[0,0],[0,0])
        sim(days_to_run,rebal,env,stat)
        ### Store some shit
    end

end

main (generic function with 2 methods)

In [180]:
env = Environment()
stat = Stats([20,20],[0,0],[0,0],[0,0])
sim(3,[20,20,12.2,24.2],env,stat)
## Seems insane, something must be wrong

([0, 40], [323, 221], [139, 99], [-76.77059249027133, -69.78552637443845], 1198)